This is perplexity code that i will be editing and using first of all.  
Basic code and Functionality     
Let's First test the code and see how it works. 



In [37]:
import threading

class Vehicle:
    def __init__(self, license_plate, vehicle_type):
        self.license_plate = license_plate
        self.vehicle_type = vehicle_type

class ParkingSpot:
    def __init__(self, spot_id):
        self.spot_id = spot_id
        self.is_occupied = False
        self.vehicle = None

class Floor:
    def __init__(self, floor_id, spots):
        self.floor_id = floor_id
        self.spots = [ParkingSpot(i) for i in range(spots)]

In [38]:
class ParkingLot:
    def __init__(self, num_floors, spots_per_floor):
        self.num_floors = num_floors
        self.spots_per_floor = spots_per_floor
        self.floors = [Floor(i, spots_per_floor) for i in range(num_floors)]
        self.vehicle_locations = {}
        self.lock = threading.Lock()

    def park_vehicle(self, license_plate, vehicle_type):
        with self.lock:
            # Find nearest available spot(s) based on vehicle type
            for floor in self.floors:
                for spot in floor.spots:
                    if not spot.is_occupied:
                        if vehicle_type == "Truck":
                            # Check for two consecutive spots
                            if spot.spot_id < len(floor.spots) - 1 and not floor.spots[spot.spot_id + 1].is_occupied:
                                spot.is_occupied = True
                                floor.spots[spot.spot_id + 1].is_occupied = True
                                vehicle = Vehicle(license_plate, vehicle_type)
                                spot.vehicle = vehicle
                                floor.spots[spot.spot_id + 1].vehicle = vehicle
                                self.vehicle_locations[license_plate] = (floor.floor_id, spot.spot_id)
                                return f"Truck {license_plate} parked at Floor {floor.floor_id}, Spot {spot.spot_id} and {spot.spot_id + 1}"
                        else:
                            spot.is_occupied = True
                            vehicle = Vehicle(license_plate, vehicle_type)
                            spot.vehicle = vehicle
                            self.vehicle_locations[license_plate] = (floor.floor_id, spot.spot_id)
                            return f"{vehicle_type} {license_plate} parked at Floor {floor.floor_id}, Spot {spot.spot_id}"
            return "Parking lot is full."

    def leave_vehicle(self, license_plate):
        with self.lock:
            if license_plate in self.vehicle_locations:
                floor_id, spot_id = self.vehicle_locations[license_plate]
                floor = self.floors[floor_id]
                spot = floor.spots[spot_id]
                spot.is_occupied = False
                # print("spot_id", spot_id)

                Vehicle = spot.vehicle.vehicle_type
                if spot.vehicle.vehicle_type == "Truck":
                    floor.spots[spot_id + 1].is_occupied = False
                    floor.spots[spot_id + 1].vehicle = None
                spot.vehicle = None
                del self.vehicle_locations[license_plate]
                return f"{Vehicle} {license_plate} has left."
            else:
                return f"Vehicle {license_plate} not found."

In [39]:
def all_vehicle_locations(parking_lot):
    return parking_lot.vehicle_locations

In [40]:
parking_lot = ParkingLot(num_floors=3, spots_per_floor=10)

In [41]:
print(parking_lot.park_vehicle("KA-01-1234", "Car"))
print(parking_lot.park_vehicle("KA-01-2345", "Truck"))
print(parking_lot.park_vehicle("KA-01-4012", "Car"))

Car KA-01-1234 parked at Floor 0, Spot 0
Truck KA-01-2345 parked at Floor 0, Spot 1 and 2
Car KA-01-4012 parked at Floor 0, Spot 3


In [ ]:
print(parking_lot.leave_vehicle("KA-01-1234"))
all_vehicle_locations(parking_lot)

Car KA-01-1234 has left.


{'KA-01-2345': (0, 1), 'KA-01-4012': (0, 3)}